In [2]:
import pandas as pd
import numpy as np
import json
import os

import torch
from torch.utils.data import TensorDataset
from torch.optim import AdamW

from tqdm.notebook import tqdm

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup # AdamW (deprec)


# 1. Fine-tune rubert-tiny2, extract embeddings and train SVM, LogitBoost, and LogitRegression

batch_size = 8, epochs = 5

In [2]:
# pretrained model
BASE_BERT = 'cointegrated/rubert-tiny2'

# data balance
major_class_exrta_w = 1.2

# data loading
max_length = int(2048*0.75)
batch_size = 8

# train
epochs = 5

#not random
seed_val = 42

#save
bert_path = f'./temp/models/v2_{batch_size}_{max_length}_{major_class_exrta_w}/'

###################
# compute on GPU #0
device_id = '0'

In [65]:
try:
    os.mkdir(bert_path)
    os.mkdir(bert_path+'epochs')
except:
    pass
try:
    os.mkdir(bert_path+'epochs')
except:
    pass

In [68]:
device = torch.device("cuda:{}".format(str(device_id)) if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=3)

# Load data

In [4]:
fold='../data/data_for_binary_classification/'
file_='chatGPT_3_instr0_withEx_temp0_train_all_updated.csv'

df=pd.read_csv(fold+file_, sep="|", encoding ='utf-8')[['text', 'final_label']] #'label_crowd', 'gpt_result', 
df=df.rename(columns={'final_label':'label'})
print (df.shape[0])
df.head()

5035


,text,label
0,"Думаете, что умеете пользоваться фотошопом?...",0.0
1,...Самое страшное - это когда ты стоишь под х...,1.0
2,Друзья мои! Поддержим дочку моей подруги! Про...,1.0
3,"Мой новый дневник, читаем, коментим :)",0.0
4,РУССКИЙ КРЫМ - МИФ для быдла! (о чем молчат ...,0.0


In [5]:
df.label = df.label.astype('int')

In [6]:
# combine 'doesn't reflect' and 'spam' classes
df.label=df.label.replace(3, 0)
df.label.value_counts()


0    3301
1    1734
Name: label, dtype: int64

## TRAIN

In [72]:
possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{0: 0, 1: 1}

In [73]:
df['label'] = df.label.replace(label_dict)

In [76]:
from sklearn.utils.class_weight import compute_class_weight
class_w = compute_class_weight('balanced', classes=df.label.unique(), y=df.label)

# fix disbalance to major class
major_class_idx = np.argmin(class_w)
class_w[major_class_idx] = class_w[major_class_idx]/major_class_exrta_w
class_w

array([0.63553974, 1.45184544])

In [77]:
X = df.text
Y = df.label
X.shape, Y.shape

((5035,), (5035,))

In [78]:
# split into train and test
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X.values, Y.values, test_size = .20, stratify = Y.values, random_state = 87)

In [79]:
X_train.shape , X_val.shape , X_train.shape[0] + X_val.shape[0]

((4028,), (1007,), 5035)

In [80]:
%%time


tokenizer = BertTokenizer.from_pretrained(BASE_BERT, 
                                          do_lower_case=False)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    X_train, 
    add_special_tokens=True, 
    return_attention_mask=True,
    return_token_type_ids=False,
    padding='max_length', # можно поставить True  #'max_length'
    truncation=True,
    max_length=max_length, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val, 
    add_special_tokens=True, 
    return_attention_mask=True,
    return_token_type_ids=False,
    padding='max_length',  #'max_length',
    truncation=True,
    max_length=max_length, 
    return_tensors='pt'
)

CPU times: user 4.74 s, sys: 128 ms, total: 4.87 s
Wall time: 5.6 s


In [81]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [82]:
model = BertForSequenceClassification.from_pretrained(BASE_BERT,
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elemen

In [83]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), #SequentialSampler
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [84]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [85]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted'), f1_score(labels_flat, preds_flat, average='macro')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} = {len(y_preds[y_preds==label])/len(y_true)}\n')

In [86]:
import random
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [87]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
#         batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels':         batch[2].to(device),
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [88]:
# save
def save_checpoint(model, optimizer, output_model):
    # save
    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()}, output_model)

# save(model, optimizer)

In [89]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(class_w), reduction = 'mean').to(device)
scores = {}

for epoch in tqdm(range(0,5)): # 1,epochs+1
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
#         batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels':         batch[2].to(device),
                 }       

        outputs = model(**inputs)
        logits = outputs['logits']
        
        
        loss = criterion(logits, inputs['labels'])
        loss_train_total += loss.item()    
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
                 
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1_w, val_f1_macro  = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted/Macro): {(val_f1_w, val_f1_macro)}')
    accuracy_per_class(predictions, true_vals)
    
    scores[epoch] = {'Training loss':loss_train_avg,
                     'Validation loss':val_loss,
                     'F1 Score (Weighted/Macro)':(val_f1_w, val_f1_macro) 
                    }
    
    # save checkpoint
    save_checpoint(model, optimizer, bert_path+f'epochs/{epoch}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 0:   0%|          | 0/504 [00:00<?, ?it/s]


Epoch 0
Training loss: 0.6455677735900122
Validation loss: 0.6234295635469376
F1 Score (Weighted/Macro): (0.656251957711443, 0.6468783185467086)
Class: 0
Accuracy: 370/660 = 0.5606060606060606

Class: 1
Accuracy: 284/347 = 0.8184438040345822



Epoch 1:   0%|          | 0/504 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.5627760663037262
Validation loss: 0.5686113704291601
F1 Score (Weighted/Macro): (0.7084040459663059, 0.6914517516401745)
Class: 0
Accuracy: 442/660 = 0.6696969696969697

Class: 1
Accuracy: 264/347 = 0.760806916426513



...


Epoch 2
Training loss: 0.5233887325794924
Validation loss: 0.556405167849291
F1 Score (Weighted/Macro): (0.7316920650851454, 0.7120496307531641)
Class: 0
Accuracy: 476/660 = 0.7212121212121212

Class: 1
Accuracy: 255/347 = 0.7348703170028819



Epoch 3:   0%|          | 0/504 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.5005263998690579
Validation loss: 0.563113215660292
F1 Score (Weighted/Macro): (0.7297478046872117, 0.7099630338745637)
Class: 0
Accuracy: 475/660 = 0.7196969696969697

Class: 1
Accuracy: 254/347 = 0.7319884726224783



Epoch 4:   0%|          | 0/504 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.4895340929340039
Validation loss: 0.5619315552333045
F1 Score (Weighted/Macro): (0.7324875421414411, 0.7124270358566647)
Class: 0
Accuracy: 479/660 = 0.7257575757575757

Class: 1
Accuracy: 253/347 = 0.729106628242075



In [ ]:
with open(bert_path+f'epochs/scores.json', 'w') as f:
    json.dump(scores, f)

# load best

best_epoch = np.argmin([scores[e]['Validation loss'] for e in scores]) + 1
print(f'best_epoch #{best_epoch}')

checkpoint = torch.load(bert_path+f'epochs/{best_epoch}', map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.save_pretrained(bert_path)
tokenizer.save_pretrained(bert_path)

In [102]:
#! SAVE MODEL !
model.save_pretrained(bert_path)
tokenizer.save_pretrained(bert_path)
bert_path

'./temp/models/v2_8_1536_1.2/'

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

## export

In [81]:
rc_path = './models/v2_8_1536_1.2/'

In [82]:
from transformers import BertModel
from transformers import BertTokenizer

In [83]:
# b_model = BertModel.from_pretrained(bert_path)
# b_model

In [84]:
# b_model.save_pretrained(rc_path)
# tokenizer.save_pretrained(rc_path)

In [85]:
model = BertModel.from_pretrained(rc_path)
tokenizer = BertTokenizer.from_pretrained(rc_path)

Some weights of the model checkpoint at ./models/v2_8_1536_1.2/ were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Take embeddings forn fine-tuned rubert-tiny2

In [87]:
model_emb = model.base_model

In [88]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [90]:
%%time
from tqdm.notebook import tqdm
emb_list = list()
it = 0
for s in tqdm(df.text.values):
    emb_list.append(embed_bert_cls(s, model_emb, tokenizer))

  0%|          | 0/5035 [00:00<?, ?it/s]

CPU times: user 28min 6s, sys: 1min 46s, total: 29min 52s
Wall time: 44.7 s


In [91]:
# embeddings for train dataset:
labels = df["label"]

emb_list_ = [np.asarray(s) for s in emb_list]
df_emb_=pd.DataFrame(emb_list_)
df_emb_

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,-0.018464,-0.025443,-0.005412,-0.067210,-0.008098,-0.042848,0.048566,0.038053,-0.051750,0.016248,...,0.013072,-0.137592,-0.063010,-0.102285,-0.017897,0.058684,-0.000319,0.078772,-0.005704,0.041218
1,0.087185,-0.013556,0.034466,-0.041398,-0.009335,0.038518,0.072062,-0.126953,0.072556,-0.010305,...,-0.009245,0.031439,0.044652,0.031633,-0.081807,-0.021026,-0.011231,0.007678,-0.057428,0.005134
2,0.121493,0.024607,-0.038873,-0.034820,-0.033760,-0.023758,0.055283,-0.082305,-0.030336,-0.063620,...,0.074011,0.065398,0.025788,0.005126,0.049230,-0.076790,0.114930,0.083010,0.026723,-0.053610
3,-0.020010,0.041242,-0.016424,-0.036992,-0.039034,0.003855,0.005490,0.042822,-0.038598,-0.078856,...,-0.016536,-0.062215,-0.011497,-0.130887,0.024420,0.049948,-0.068521,-0.050156,0.030229,0.039134
4,0.026711,-0.061871,0.018863,-0.026486,0.046739,-0.060128,0.027036,0.004546,-0.053569,0.005665,...,-0.000268,-0.096092,-0.059798,-0.059040,0.025562,0.083468,0.007707,0.097353,0.100729,-0.048607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,0.090720,0.068379,0.007549,-0.037703,0.008094,-0.022078,0.057416,0.074114,0.012745,0.036362,...,0.039732,-0.056965,-0.118393,0.032156,0.018497,0.021067,0.037554,0.101765,0.023785,0.004840
5031,0.143055,0.031883,0.000566,-0.107946,-0.004803,-0.010487,0.043309,0.042991,-0.025998,0.029961,...,0.030708,0.031169,0.063458,-0.007555,-0.065752,0.068371,-0.064800,0.034165,0.053517,-0.010068
5032,0.124135,0.004982,-0.001954,-0.043581,-0.027300,-0.040936,0.006083,-0.125258,-0.009013,-0.044418,...,0.051877,0.048784,0.106491,0.048471,-0.019057,-0.011693,0.046124,-0.033521,0.045135,-0.047027
5033,0.050855,0.037297,0.013623,-0.040512,-0.012929,-0.042989,0.068927,0.009016,0.003902,0.060874,...,0.009632,-0.044231,0.059576,-0.017738,-0.072191,-0.035146,-0.112178,0.042907,-0.006640,-0.064849


In [16]:
# Load trial dataset

file_test='all_merged_temp0_instr0_withEx_test_final_label.csv'

test_data=pd.read_csv(fold+file_test, sep="|", encoding ='utf-8')[['text', 'final_label']]
test_data=test_data.rename(columns={'final_label':'label_test'})
print (test_data.shape[0])
test_data.head()

804


,text,label_test
0,"- интересный новый сервис, где можно оставить...",1
1,чет как-то нерадостно все это...особо на фоне...,0
2,#Repost with . ・・・ жаль что быстро убежала!!!#...,0
3,#hellomyearth #дорогажизни #разорванноекольцо,0
4,#ВтандемеСМамой#кактампробка#😁,0


In [17]:
test_data.replace({'label_test': {3: 0}}, inplace=True)
test_data.label_test.value_counts()

0    532
1    272
Name: label_test, dtype: int64

In [94]:
%%time
from tqdm.notebook import tqdm
emb_list_test = list()
it = 0
for s in tqdm(test_data.text.values):
    emb_list_test.append(embed_bert_cls(s, model_emb, tokenizer))

  0%|          | 0/804 [00:00<?, ?it/s]

CPU times: user 4min 16s, sys: 15.8 s, total: 4min 32s
Wall time: 6.81 s


In [95]:
# embeddings for trial dataset:
    
labels_test=test_data['label_test']
emb_list_test_ = [np.asarray(s) for s in emb_list_test]
df_emb_test_=pd.DataFrame(emb_list_test_)
df_emb_test_

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,-0.047982,-0.007989,0.019273,-0.064445,-0.047470,0.079242,-0.094084,0.037180,0.049691,-0.039148,...,0.050658,-0.002591,-0.030989,0.011156,-0.025440,0.075424,0.062284,0.021909,0.038568,-0.067602
1,0.048708,0.038768,0.008993,-0.043480,-0.011525,-0.074801,0.068448,-0.029761,0.038171,-0.086254,...,0.015159,-0.086703,0.058923,-0.156870,0.034332,0.029744,-0.051479,0.020975,-0.005021,-0.045287
2,0.046934,-0.007776,-0.034438,-0.060966,0.001300,-0.036679,0.076556,-0.046489,-0.026749,-0.019772,...,0.004219,-0.030804,-0.031761,-0.056468,0.068399,0.162177,-0.056490,0.092742,0.047726,-0.021333
3,-0.044973,-0.031754,0.013514,-0.088200,0.005767,0.006182,0.086622,0.048924,-0.041500,-0.050908,...,-0.058033,-0.037874,-0.066198,-0.072266,0.083422,0.040231,-0.079380,0.028450,0.032081,0.027410
4,0.019812,0.003509,0.010675,-0.081504,-0.021689,-0.016962,0.031124,0.072162,-0.058702,-0.041161,...,-0.032649,-0.088169,-0.050347,-0.059466,0.042321,0.087201,-0.081285,0.027309,0.088323,-0.029691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,0.101509,-0.086583,0.037290,-0.097883,-0.067217,-0.012579,0.081658,0.014439,-0.060402,-0.037441,...,-0.020561,-0.054927,0.003964,-0.058199,0.079025,0.088163,0.000104,0.089325,0.038812,0.045627
800,0.015678,-0.010740,-0.001362,-0.025951,-0.016946,0.006844,-0.015210,0.062379,0.035687,-0.057801,...,-0.027969,-0.004778,-0.050198,-0.040739,0.081978,0.084395,0.013094,0.002331,0.059930,-0.015127
801,0.017429,-0.018824,0.004258,-0.043123,0.043978,-0.012925,-0.038624,0.043852,-0.117673,0.061700,...,0.001252,-0.078440,-0.072294,-0.057007,0.068025,0.063575,-0.018018,0.005897,0.067275,-0.090942
802,-0.004265,0.044574,0.046853,-0.093753,-0.026876,0.010413,-0.004832,0.034466,-0.071080,0.073750,...,0.043476,-0.076574,0.039202,0.005576,-0.015504,0.040497,-0.113724,0.067692,0.015220,-0.046457


In [ ]:
# embeddings were pre-saved:
# fold='../data/data_for_binary_classification/'
# df_emb_.to_csv(fold+'embeddings312_RuTiny2_train.csv', sep='|', encoding='utf-8', index=False)
# df_emb_test_.to_csv(fold+'embeddings312_RuTiny2_test.csv', sep='|', encoding='utf-8', index=False)

# so we can just load them:
# df_emb_loaded=pd.read_csv(fold+'embeddings312_RuTiny2_train.csv', sep='|', encoding='utf-8')
# df_emb_loaded_test=pd.read_csv(fold+'embeddings312_RuTiny2_test.csv', sep='|', encoding='utf-8')

# SVM

In [96]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

import numpy as np
class_weight = compute_class_weight(
    class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight

array([0.76264768, 1.45184544])

In [97]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameteres = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
clf = GridSearchCV(SVC(class_weight={0:class_weight[0], 1:class_weight[1]}, probability=True), param_grid=parameteres , cv=5, scoring='f1_macro')
clf.fit(df_emb_, labels)

GridSearchCV(cv=5,
             estimator=SVC(class_weight={0: 0.7626476825204483,
                                         1: 1.4518454440599768},
                           probability=True),
             param_grid={'C': [1], 'gamma': [1], 'kernel': ['poly']},
             scoring='f1_macro')

In [98]:
print("Best parameters from gridsearch: {}".format(clf.best_params_))
print("CV score=%0.3f" % clf.best_score_)
cv_results = clf.cv_results_
print(cv_results)

Best parameters from gridsearch: {'C': 1, 'gamma': 1, 'kernel': 'poly'}
CV score=0.759
{'mean_fit_time': array([7.82658319]), 'std_fit_time': array([0.40208642]), 'mean_score_time': array([0.28000245]), 'std_score_time': array([0.01259353]), 'param_C': masked_array(data=[1],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[1],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['poly'],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'C': 1, 'gamma': 1, 'kernel': 'poly'}], 'split0_test_score': array([0.82165679]), 'split1_test_score': array([0.65893225]), 'split2_test_score': array([0.73444093]), 'split3_test_score': array([0.84713307]), 'split4_test_score': array([0.73153275]), 'mean_test_score': array([0.75873916]), 'std_test_score': array([0.06791778]), 'rank_test_score': array([1], dtype=int32)}


In [99]:
y_predict=clf.predict_proba(df_emb_test_)

In [100]:
import itertools
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def TestCutoff(df):

    cut_off_list_=np.arange(0.005, 0.901, 0.005)
#     cut_off_list = itertools.chain([0.01], cut_off_list_)
    
    f1score_macro_list=[]
    f1score_list=[]
    recall_list=[]
    
    predict_list_list=[[]]
    for i, cut_off in enumerate(cut_off_list_):
        predict_list=np.where(df['predict_1']>cut_off, 1, 0)
        predict_list_list.append(predict_list)
        print (cut_off)
        precision, recall, f1score = precision_recall_fscore_support(df['label_test'], predict_list)[:3]
        print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')
        f1score_list.append(f1score[1])
        recall_list.append(recall[1])
        print ("macro:")
        precision, recall, f1score_macro = precision_recall_fscore_support(df['label_test'], predict_list, average='macro')[:3]
        print(f'precision: {precision}, recall: {recall}, f1score_macro: {f1score_macro}')
        f1score_macro_list.append(f1score_macro)
        print (" ")
    
    max_ind=f1score_macro_list.index(max(f1score_macro_list))   
    print ("macro", f1score_macro_list[max_ind])
    print ("F1-valued:", f1score_list[max_ind])
    print ("recall-valued:", recall_list[max_ind])
    print (cut_off_list_[max_ind])
    
    df['predict']=predict_list_list[max_ind+1]
        
    return (df)
#         
    

In [ ]:
predictions0=[]
predictions1=[]

for res in y_predict:
    predictions0.append(res[0])
    predictions1.append(res[1])

test_data['predict_0']=predictions0
test_data['predict_1']=predictions1

test_data_predict=TestCutoff(test_data)

In [102]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1score = precision_recall_fscore_support(test_data.label_test, test_data['predict'])[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.875      0.78461538], recall: [0.89473684 0.75      ], f1score: [0.88475836 0.76691729]


In [103]:
precision, recall, f1score = precision_recall_fscore_support(test_data.label_test, test_data['predict'], average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score_macro: {f1score}')

precision: 0.8298076923076922, recall: 0.8223684210526316, f1score_macro: 0.8258378287726752


# Prediction for the main dataset

In [25]:
# This is the best model
# Load main dataset:
f='../data/'
file='data_for_labeling_cleaned.csv'
data_full=pd.read_csv(f+file, sep="|", encoding ='utf-8')[['text']]
data_full

,text
0,Анамалия близ Чернобыля
1,и кто осмелится прекратить этот разбой? откров...
2,"Никогда не выравайся от парня, если не уверенн..."
3,Ураааа)))Я опять у Риты.
4,"Может, она и приврала. Может, она и слабохарак..."
...,...
73276,Наши рубашки представлены на благотворительном...
73277,Айка итии ба5айы дьи!
73278,и все великие когда-то делали первые шаги на п...
73279,#Repost with . ・・・ USSR ballet The Queens proj...


In [116]:
%%time
from tqdm.notebook import tqdm
emb_list_full = list()
it = 0
for s in tqdm(data_full['text']):
    emb_list_full.append(embed_bert_cls(s, model_emb, tokenizer))

  0%|          | 0/73285 [00:00<?, ?it/s]

CPU times: user 6h 31min 24s, sys: 23min 34s, total: 6h 54min 58s
Wall time: 10min 14s


In [117]:
y_predict_full=clf.predict_proba(emb_list_full)

In [119]:
predictions0=[]
predictions1=[]


for res in y_predict_full:
    predictions0.append(res[0])
    predictions1.append(res[1])

    
data_full['predict_0']=predictions0
data_full['predict_1']=predictions1
data_full

,text,predict_0,predict_1
0,Анамалия близ Чернобыля,0.780301,0.219699
1,и кто осмелится прекратить этот разбой? откров...,0.789129,0.210871
2,"Никогда не выравайся от парня, если не уверенн...",0.400666,0.599334
3,Ураааа)))Я опять у Риты.,0.943565,0.056435
4,"Может, она и приврала. Может, она и слабохарак...",0.180333,0.819667
...,...,...,...
73280,Наши рубашки представлены на благотворительном...,0.801630,0.198370
73281,Айка итии ба5айы дьи!,0.936358,0.063642
73282,и все великие когда-то делали первые шаги на п...,0.343873,0.656127
73283,#Repost with . ・・・ USSR ballet The Queens proj...,0.940227,0.059773


In [120]:
cut_off=0.420
predict_list=[]
predict_list=np.where(data_full['predict_1']>cut_off, 1, 0)
predict_list

array([0, 0, 1, ..., 1, 0, 1])

In [121]:
data_full['predict']=predict_list
data_full.predict.value_counts()

0    49985
1    23300
Name: predict, dtype: int64

In [127]:
data_full.predict.value_counts(normalize=True)

0    0.682063
1    0.317937
Name: predict, dtype: float64

In [126]:
data_full.to_csv(f+'predicted_73285_2classes', sep='|', encoding='utf-8', index=False)

# Logit Boost

In [33]:
from logitboost import LogitBoost
from sklearn.model_selection import GridSearchCV

In [34]:
steps = [('LogitBoost', LogitBoost())]
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps) # define the pipeline object.

parameteres = {'LogitBoost__n_estimators':range(10,100,10)}
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)


In [35]:
grid.fit(df_emb_, labels)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('LogitBoost', LogitBoost())]),
             param_grid={'LogitBoost__n_estimators': range(10, 100, 10)})

In [36]:
print("Best parameters from gridsearch: {}".format(grid.best_params_))
print("CV score=%0.3f" % grid.best_score_)
cv_results = grid.cv_results_
print(cv_results)

Best parameters from gridsearch: {'LogitBoost__n_estimators': 30}
CV score=0.775
{'mean_fit_time': array([1.09624925, 2.16360064, 3.18853326, 4.23988729, 5.18478107,
       6.06775427, 7.22547455, 8.31915588, 9.06779461]), 'std_fit_time': array([0.03912988, 0.04561217, 0.01365214, 0.03828939, 0.08121675,
       0.06951414, 0.09377665, 0.033249  , 0.11800694]), 'mean_score_time': array([0.02238917, 0.04348617, 0.07130294, 0.0893476 , 0.10918365,
       0.12896137, 0.15132327, 0.17679143, 0.19598117]), 'std_score_time': array([0.0002611 , 0.0002335 , 0.00690478, 0.00595087, 0.00303086,
       0.00455219, 0.00535931, 0.0053402 , 0.00933154]), 'param_LogitBoost__n_estimators': masked_array(data=[10, 20, 30, 40, 50, 60, 70, 80, 90],
             mask=[False, False, False, False, False, False, False, False,
                   False],
       fill_value='?',
            dtype=object), 'params': [{'LogitBoost__n_estimators': 10}, {'LogitBoost__n_estimators': 20}, {'LogitBoost__n_estimators': 30

In [37]:
y_pred = grid.predict_proba(df_emb_test_)

In [ ]:
predictions0=[]
predictions1=[]


for res in y_pred:
    predictions0.append(res[0])
    predictions1.append(res[1])

    
test_data['predict_0']=predictions0
test_data['predict_1']=predictions1

test_data_predict=TestCutoff(test_data)

In [44]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1score = precision_recall_fscore_support(test_data.label_test, test_data['predict'])[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.87406015 0.75367647], recall: [0.87406015 0.75367647], f1score: [0.87406015 0.75367647]


In [45]:
precision, recall, f1score = precision_recall_fscore_support(test_data.label_test, test_data['predict'], average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score_macro: {f1score}')

precision: 0.8138683104820876, recall: 0.8138683104820876, f1score_macro: 0.8138683104820876


# Logit Regression

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
import numpy as np
class_weight = compute_class_weight(
    class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight

array([0.76264768, 1.45184544])

In [43]:
parameters = {'C': np.linspace(0.0001, 10, 50), "penalty":["l1","l2"]}  #high C means "Trust this training data a lot", while a low value says "This data may not be fully representative of the real world data, so if it's telling you to make a parameter really large, don't listen to it"
grid_search = GridSearchCV(LogisticRegression(class_weight={0:class_weight[0], 1:class_weight[1]}), parameters, cv=5, scoring='f1_macro')
grid_search.fit(df_emb_, labels)

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight={0: 0.7626476825204483,
                                                        1: 1.4518454440599768}),
             param_grid={'C': array([1.00000000e-04, 2.04179592e-01, 4.08259184e-01, 6.12338776e-01,
       8.16418367e-01, 1.02049796e+00, 1.22457755e+00, 1.42865714e+00,
       1.63273673e+00, 1.83681633e+00, 2.04089592e+00, 2.24497551e+00,
       2.44905510e+00, 2.65313469e+00, 2.85721429e+...
       5.71432857e+00, 5.91840816e+00, 6.12248776e+00, 6.32656735e+00,
       6.53064694e+00, 6.73472653e+00, 6.93880612e+00, 7.14288571e+00,
       7.34696531e+00, 7.55104490e+00, 7.75512449e+00, 7.95920408e+00,
       8.16328367e+00, 8.36736327e+00, 8.57144286e+00, 8.77552245e+00,
       8.97960204e+00, 9.18368163e+00, 9.38776122e+00, 9.59184082e+00,
       9.79592041e+00, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [47]:
print("Best parameters from gridsearch: {}".format(grid_search.best_params_))
print("CV score=%0.3f" % grid_search.best_score_)
cv_resultsl = grid_search.cv_results_
print(cv_resultsl)

Best parameters from gridsearch: {'C': 0.0001, 'penalty': 'l2'}
CV score=0.752
{'mean_fit_time': array([0.0037147 , 0.0241519 , 0.00322347, 0.03687806, 0.00352597,
       0.04486532, 0.0034966 , 0.05093918, 0.00346422, 0.05576649,
       0.00345654, 0.05837436, 0.00339322, 0.05948205, 0.00349436,
       0.06587572, 0.00339398, 0.0695591 , 0.0031878 , 0.07109642,
       0.00351248, 0.07362995, 0.00322194, 0.07169089, 0.00315657,
       0.07404079, 0.00317473, 0.07627969, 0.00327497, 0.07827225,
       0.00326552, 0.08168635, 0.00330272, 0.08327537, 0.00328941,
       0.08828707, 0.00332866, 0.0960856 , 0.0032742 , 0.09341364,
       0.00320144, 0.10248451, 0.00319657, 0.09284182, 0.00321774,
       0.09870801, 0.00321226, 0.09357629, 0.00322189, 0.09482098,
       0.00321736, 0.12806931, 0.0033103 , 0.10588021, 0.00331531,
       0.10088758, 0.00330186, 0.10186496, 0.00327821, 0.09370041,
       0.00323653, 0.09430008, 0.00324974, 0.09446816, 0.00323725,
       0.09610448, 0.00325623, 0

In [48]:
y_pred = grid_search.predict_proba(df_emb_test_)

In [ ]:
predictions0=[]
predictions1=[]


for res in y_pred:
    predictions0.append(res[0])
    predictions1.append(res[1])

    
test_data['predict_0']=predictions0
test_data['predict_1']=predictions1

test_data_predict=TestCutoff(test_data)

In [50]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1score = precision_recall_fscore_support(test_data.label_test, test_data['predict'])[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.87977099 0.74642857], recall: [0.86654135 0.76838235], f1score: [0.87310606 0.75724638]


In [51]:
precision, recall, f1score = precision_recall_fscore_support(test_data.label_test, test_data['predict'], average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score_macro: {f1score}')

precision: 0.8130997818974919, recall: 0.8174618531623176, f1score_macro: 0.8151762187088274


# 2. Train models based on 768-embeddings from rubert-base-cased-conversational

In [52]:
from transformers import AutoTokenizer, AutoModel 
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational", model_max_length=512) 
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational", num_labels=2)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [54]:
%%time
from tqdm.notebook import tqdm
emb_list = list()
it = 0
for s in tqdm(df.text):
    emb_list.append(embed_bert_cls(s, model, tokenizer))

  0%|          | 0/5035 [00:00<?, ?it/s]

CPU times: user 1h 49min 11s, sys: 29.2 s, total: 1h 49min 41s
Wall time: 2min 17s


In [55]:

%%time
from tqdm.notebook import tqdm
emb_list_test = list()
it = 0
for s in tqdm(test_data.text):
    emb_list_test.append(embed_bert_cls(s, model, tokenizer))
    
    

  0%|          | 0/804 [00:00<?, ?it/s]

CPU times: user 15min 33s, sys: 2.04 s, total: 15min 35s
Wall time: 19.5 s


In [56]:
labels = df["label"]

emb_list_ = [np.asarray(s) for s in emb_list]
df_emb_=pd.DataFrame(emb_list_)
df_emb_

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.018975,-0.001351,-0.036334,0.015108,0.000549,0.070605,0.066554,0.029897,-0.054324,0.017511,...,0.016930,-0.002973,-0.052835,0.018066,0.025775,0.002032,0.038878,0.027940,0.015232,-0.037016
1,0.019812,-0.028492,-0.041963,0.012508,0.050236,-0.015364,-0.011181,0.004569,0.040616,-0.001826,...,0.007038,0.053429,-0.044758,-0.001870,0.011583,0.002814,-0.034485,-0.001803,0.025618,-0.038578
2,-0.012166,0.011635,-0.017159,0.002655,0.044597,-0.002403,-0.000588,-0.000590,0.006924,-0.033347,...,-0.000399,-0.016995,-0.054655,0.000128,-0.000498,-0.014621,0.002692,-0.017654,0.016672,0.000137
3,-0.025476,-0.018142,-0.087044,0.013581,0.045457,0.025133,-0.006145,0.012807,-0.035447,0.023708,...,0.007615,0.026365,-0.060281,-0.017085,0.006862,-0.012635,0.010918,-0.003479,0.002557,-0.004744
4,-0.067215,-0.015028,-0.102502,0.009260,0.062075,0.012592,0.008576,0.021265,-0.046282,0.038958,...,0.020768,0.039588,-0.055771,-0.049322,0.053859,-0.026940,0.021567,0.000866,0.027638,-0.065772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,-0.016395,-0.030579,-0.011700,-0.002913,-0.001894,0.033167,0.025396,-0.002383,-0.084536,0.012031,...,0.021289,0.033067,-0.025002,-0.007566,0.049071,-0.008988,0.049975,0.048090,0.001161,-0.061774
5031,-0.002448,-0.005312,0.010221,0.041342,0.038213,0.014441,-0.044771,0.064974,-0.050657,0.013742,...,0.009452,0.069418,-0.041314,-0.028628,0.010116,-0.002661,0.040237,0.023575,0.012390,-0.065933
5032,-0.049334,0.000191,-0.075015,0.012169,0.025788,0.034515,-0.013789,0.051053,-0.015268,0.065273,...,-0.036013,0.036438,-0.067699,-0.045244,0.013065,-0.000260,-0.018512,0.005339,0.040650,-0.019662
5033,0.008010,0.003892,0.034859,-0.024992,-0.019697,0.008717,-0.005210,0.037962,-0.004088,0.024534,...,0.032225,-0.013915,-0.017380,-0.023246,0.010574,-0.000123,0.009516,-0.019785,-0.003043,-0.040094


In [57]:
labels_test=test_data['label_test']

emb_list_test_ = [np.asarray(s) for s in emb_list_test]
df_emb_test_=pd.DataFrame(emb_list_test_)
df_emb_test_

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.054716,-0.045481,-0.023535,-0.000155,-0.006323,0.047523,-0.025077,-0.003400,-0.032581,-0.009139,...,-0.010738,0.006308,-0.068138,-0.055364,0.028434,-0.015068,-0.015639,-0.006466,0.031062,-0.048617
1,-0.028712,-0.043620,-0.051715,-0.027764,0.080676,-0.039356,-0.046797,0.057568,-0.024146,-0.034063,...,0.023295,0.058056,-0.054193,-0.101050,0.006665,-0.029989,-0.019366,-0.080597,-0.002285,-0.030322
2,-0.016238,-0.010530,0.011108,-0.004964,0.079042,0.044689,-0.043167,-0.001452,-0.027746,0.011876,...,-0.020584,0.043124,-0.041857,0.006142,-0.021689,-0.022326,0.004118,0.018608,0.052731,-0.060148
3,0.066568,0.027526,-0.023566,-0.004859,0.039345,0.018339,-0.058902,0.016316,-0.032561,-0.021920,...,-0.002746,0.079601,-0.076817,-0.012250,0.004977,0.024742,-0.002690,0.041313,0.074336,-0.044611
4,0.019350,0.001813,-0.006678,-0.002628,0.065334,0.033174,-0.011546,0.049876,-0.026978,-0.001274,...,-0.018426,0.044450,-0.089290,0.004893,0.029819,-0.006972,0.000377,-0.013762,0.091146,-0.035349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,-0.033637,-0.005506,0.038416,-0.018379,0.051197,0.009491,-0.017781,0.052079,-0.037465,-0.013878,...,0.016810,0.075761,-0.067668,-0.011157,-0.031862,-0.024369,0.043494,0.009058,0.051453,-0.036159
800,-0.028547,-0.045570,-0.068305,-0.007125,0.030637,0.000143,0.019804,0.057383,-0.019331,0.030423,...,-0.014081,-0.007704,-0.040679,0.018558,0.017199,0.000757,0.007542,0.022014,0.029671,0.006060
801,-0.025502,-0.001768,0.025397,0.016272,-0.003197,0.029201,0.000391,-0.004197,-0.005269,0.046244,...,-0.017122,-0.004484,-0.029274,-0.000227,-0.028656,-0.014738,0.026732,0.028872,-0.014812,-0.007151
802,-0.035723,0.011342,0.102803,-0.032850,-0.006633,-0.002737,-0.024466,0.003412,-0.032806,0.058926,...,0.002785,0.057524,-0.003627,-0.007116,0.034572,0.005385,0.015048,-0.005996,0.033237,-0.015909


In [58]:
import numpy as np
class_weight = compute_class_weight(
    class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight

array([0.76264768, 1.45184544])

In [59]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameteres = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
clf = GridSearchCV(SVC(class_weight={0:class_weight[0], 1:class_weight[1]}, probability=True), param_grid=parameteres , cv=5, scoring='f1_macro')
clf.fit(df_emb_, labels)

GridSearchCV(cv=5,
             estimator=SVC(class_weight={0: 0.7626476825204483,
                                         1: 1.4518454440599768},
                           probability=True),
             param_grid={'C': [10], 'gamma': [1], 'kernel': ['rbf']},
             scoring='f1_macro')

In [60]:
print("Best parameters from gridsearch: {}".format(clf.best_params_))
print("CV score=%0.3f" % clf.best_score_)
cv_results = clf.cv_results_
print(cv_results)

Best parameters from gridsearch: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
CV score=0.697
{'mean_fit_time': array([25.33409033]), 'std_fit_time': array([1.52929982]), 'mean_score_time': array([1.46485128]), 'std_score_time': array([0.12510906]), 'param_C': masked_array(data=[10],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[1],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['rbf'],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'C': 10, 'gamma': 1, 'kernel': 'rbf'}], 'split0_test_score': array([0.77092125]), 'split1_test_score': array([0.58629813]), 'split2_test_score': array([0.69376972]), 'split3_test_score': array([0.774876]), 'split4_test_score': array([0.65853477]), 'mean_test_score': array([0.69687997]), 'std_test_score': array([0.07109693]), 'rank_test_score': array([1], dtype=int32)}


In [61]:
y_predict=clf.predict_proba(df_emb_test_)

In [62]:
predictions0=[]
predictions1=[]

for res in y_predict:
    predictions0.append(res[0])
    predictions1.append(res[1])
    
test_data['predict_0']=predictions0
test_data['predict_1']=predictions1

test_data_predict=TestCutoff(test_data)

0.005
precision: [0.         0.33830846], recall: [0. 1.], f1score: [0.         0.50557621]
macro:
precision: 0.1691542288557214, recall: 0.5, f1score_macro: 0.2527881040892193
 
0.01
precision: [0.         0.33830846], recall: [0. 1.], f1score: [0.         0.50557621]
macro:
precision: 0.1691542288557214, recall: 0.5, f1score_macro: 0.2527881040892193
 
0.015
precision: [1.         0.33957553], recall: [0.0056391 1.       ], f1score: [0.01121495 0.50698975]
macro:
precision: 0.6697877652933832, recall: 0.5028195488721805, f1score_macro: 0.2591023508200434
 
0.02
precision: [1.         0.34042553], recall: [0.0093985 1.       ], f1score: [0.01862197 0.50793651]
macro:
precision: 0.6702127659574468, recall: 0.5046992481203008, f1score_macro: 0.2632792409328722
 
0.025
precision: [1.         0.34170854], recall: [0.01503759 1.        ], f1score: [0.02962963 0.5093633 ]
macro:
precision: 0.6708542713567839, recall: 0.5075187969924813, f1score_macro: 0.26949646275488975
 
0.030000000000000

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

precision: 0.7856444118366348, recall: 0.790579389650597, f1score_macro: 0.7879558018619115
 
0.38
precision: [0.85931559 0.71223022], recall: [0.84962406 0.72794118], f1score: [0.85444234 0.72      ]
macro:
precision: 0.7857729025904752, recall: 0.788782618310482, f1score_macro: 0.7872211720226843
 
0.385
precision: [0.85499058 0.71428571], recall: [0.85338346 0.71691176], f1score: [0.85418627 0.71559633]
macro:
precision: 0.7846381490449288, recall: 0.7851476116762495, f1score_macro: 0.7848912977810766
 
0.39
precision: [0.84972171 0.72075472], recall: [0.86090226 0.70220588], f1score: [0.85527544 0.7113594 ]
macro:
precision: 0.785238211922848, recall: 0.7815540689960194, f1score_macro: 0.7833174238037859
 
0.395
precision: [0.84587156 0.72586873], recall: [0.86654135 0.69117647], f1score: [0.85608171 0.70809793]
macro:
precision: 0.7858701427508767, recall: 0.7788589119858469, f1score_macro: 0.782089818443154
 
0.4
precision: [0.84489051 0.73046875], recall: [0.87030075 0.6875    ]

precision: [0.69050715 0.97142857], recall: [0.9981203 0.125    ], f1score: [0.81629516 0.22149837]
macro:
precision: 0.8309678617871076, recall: 0.5615601503759399, f1score_macro: 0.518896764453302
 
0.865
precision: [0.68961039 0.97058824], recall: [0.9981203  0.12132353], f1score: [0.8156682  0.21568627]
macro:
precision: 0.8300993124522535, recall: 0.5597219150818222, f1score_macro: 0.5156772386373905
 
0.87
precision: [0.68871595 0.96969697], recall: [0.9981203  0.11764706], f1score: [0.81504221 0.20983607]
macro:
precision: 0.8292064615021814, recall: 0.5578836797877046, f1score_macro: 0.5124391379288653
 
0.875
precision: [0.68516129 0.96551724], recall: [0.9981203  0.10294118], f1score: [0.81254782 0.18604651]
macro:
precision: 0.8253392658509455, recall: 0.550530738611234, f1score_macro: 0.49929716553086245
 
0.88
precision: [0.68339768 0.96296296], recall: [0.9981203  0.09558824], f1score: [0.81130634 0.17391304]
macro:
precision: 0.8231803231803232, recall: 0.546854268022998

In [63]:
precision, recall, f1score = precision_recall_fscore_support(labels_test, test_data_predict['predict'])[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.83361921 0.7918552 ], recall: [0.91353383 0.64338235], f1score: [0.87174888 0.70993915]


In [64]:
precision, recall, f1score = precision_recall_fscore_support(labels_test, test_data_predict['predict'], average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score_macro: {f1score}')

precision: 0.8127372072988055, recall: 0.7784580937638214, f1score_macro: 0.7908440134983945


# Logit Boost

In [65]:
steps = [('LogitBoost', LogitBoost())]
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps) # define the pipeline object.

parameteres = {'LogitBoost__n_estimators':range(10,100,10)}

grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)

In [66]:
grid.fit(df_emb_, labels)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('LogitBoost', LogitBoost())]),
             param_grid={'LogitBoost__n_estimators': range(10, 100, 10)})

In [67]:
print("Best parameters from gridsearch: {}".format(grid.best_params_))
print("CV score=%0.3f" % grid.best_score_)
cv_results = grid.cv_results_
print(cv_results)

Best parameters from gridsearch: {'LogitBoost__n_estimators': 70}
CV score=0.715
{'mean_fit_time': array([ 2.79942312,  5.40478673,  8.11945777, 10.54189849, 13.23489637,
       15.53023572, 18.09441257, 20.96763592, 23.50913448]), 'std_fit_time': array([0.03142028, 0.03856336, 0.10592107, 0.12890112, 0.20902708,
       0.15737183, 0.22910931, 0.28103298, 0.63385288]), 'mean_score_time': array([0.04417458, 0.08697071, 0.13355494, 0.17791805, 0.22635398,
       0.26530986, 0.32098236, 0.36329894, 0.38717256]), 'std_score_time': array([0.00176943, 0.00594447, 0.01050529, 0.00997894, 0.01408817,
       0.00639642, 0.03723074, 0.02172834, 0.01079422]), 'param_LogitBoost__n_estimators': masked_array(data=[10, 20, 30, 40, 50, 60, 70, 80, 90],
             mask=[False, False, False, False, False, False, False, False,
                   False],
       fill_value='?',
            dtype=object), 'params': [{'LogitBoost__n_estimators': 10}, {'LogitBoost__n_estimators': 20}, {'LogitBoost__n_estima

In [68]:
y_pred = grid.predict_proba(df_emb_test_)

In [69]:
predictions0=[]
predictions1=[]


for res in y_pred:
    predictions0.append(res[0])
    predictions1.append(res[1])

    
test_data['predict_0']=predictions0
test_data['predict_1']=predictions1

test_data_predict=TestCutoff(test_data)

0.005
precision: [0.96      0.3478819], recall: [0.04511278 0.99632353], f1score: [0.08617594 0.51569933]
macro:
precision: 0.6539409499358151, recall: 0.520718155683326, f1score_macro: 0.3009376382585108
 
0.01
precision: [0.93548387 0.36118598], recall: [0.10902256 0.98529412], f1score: [0.1952862  0.52859961]
macro:
precision: 0.6483349273976176, recall: 0.5471583370190182, f1score_macro: 0.3619429004044389
 
0.015
precision: [0.93617021 0.37464789], recall: [0.16541353 0.97794118], f1score: [0.28115016 0.54175153]
macro:
precision: 0.6554090500449505, recall: 0.5716773551525873, f1score_macro: 0.4114508436196586
 
0.02
precision: [0.94642857 0.38439306], recall: [0.19924812 0.97794118], f1score: [0.32919255 0.55186722]
macro:
precision: 0.6654108175061932, recall: 0.58859464838567, f1score_macro: 0.44052988325043174
 
0.025
precision: [0.94244604 0.39699248], recall: [0.2462406  0.97058824], f1score: [0.390462   0.56350053]
macro:
precision: 0.6697192621842376, recall: 0.6084144183

precision: [0.82300885 0.71966527], recall: [0.87406015 0.63235294], f1score: [0.84776664 0.67318982]
macro:
precision: 0.7713370607620247, recall: 0.7532065457762052, f1score_macro: 0.7604782300777606
 
0.425
precision: [0.82394366 0.72881356], recall: [0.87969925 0.63235294], f1score: [0.85090909 0.67716535]
macro:
precision: 0.7763786106469324, recall: 0.7560260946483857, f1score_macro: 0.7640372226198998
 
0.43
precision: [0.82249561 0.72765957], recall: [0.87969925 0.62867647], f1score: [0.85013624 0.67455621]
macro:
precision: 0.7750775903974871, recall: 0.754187859354268, f1score_macro: 0.7623462263998839
 
0.435
precision: [0.81993007 0.72844828], recall: [0.88157895 0.62132353], f1score: [0.84963768 0.67063492]
macro:
precision: 0.7741891728960695, recall: 0.7514512383900929, f1score_macro: 0.7601363008971704
 
0.44
precision: [0.82086957 0.73799127], recall: [0.88721805 0.62132353], f1score: [0.85275519 0.6746507 ]
macro:
precision: 0.7794304157964687, recall: 0.7542707872622

precision: [0.7532656  0.88695652], recall: [0.97556391 0.375     ], f1score: [0.85012285 0.52713178]
macro:
precision: 0.8201110620306682, recall: 0.675281954887218, f1score_macro: 0.6886273165342933
 
0.74
precision: [0.75397974 0.90265487], recall: [0.97932331 0.375     ], f1score: [0.85200327 0.52987013]
macro:
precision: 0.8283173033822984, recall: 0.6771616541353384, f1score_macro: 0.6909367002580412
 
0.745
precision: [0.75397974 0.90265487], recall: [0.97932331 0.375     ], f1score: [0.85200327 0.52987013]
macro:
precision: 0.8283173033822984, recall: 0.6771616541353384, f1score_macro: 0.6909367002580412
 
0.75
precision: [0.75180375 0.9009009 ], recall: [0.97932331 0.36764706], f1score: [0.85061224 0.52219321]
macro:
precision: 0.8263523263523264, recall: 0.6734851835471031, f1score_macro: 0.6864027281931049
 
0.755
precision: [0.75216138 0.90909091], recall: [0.98120301 0.36764706], f1score: [0.85154976 0.52356021]
macro:
precision: 0.8306261461881058, recall: 0.6744250331711

In [70]:
precision, recall, f1score = precision_recall_fscore_support(test_data.label_test, test_data['predict'])[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.82068966 0.75      ], recall: [0.89473684 0.61764706], f1score: [0.85611511 0.67741935]


In [71]:
precision, recall, f1score = precision_recall_fscore_support(labels_test, test_data_predict['predict'], average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score_macro: {f1score}')

precision: 0.7853448275862069, recall: 0.7561919504643964, f1score_macro: 0.7667672313761894


# Logit Regression

In [72]:
parameters = {'C': np.linspace(0.0001, 10, 50), "penalty":["l1","l2"]}  #high C means "Trust this training data a lot", while a low value says "This data may not be fully representative of the real world data, so if it's telling you to make a parameter really large, don't listen to it"
grid_search = GridSearchCV(LogisticRegression(class_weight={0:class_weight[0], 1:class_weight[1]}), parameters, cv=5, scoring='f1_macro')
grid_search.fit(df_emb_, labels)

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

...

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mmilkov2/miniconda3/envs/TextMining_py3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight={0: 0.7626476825204483,
                                                        1: 1.4518454440599768}),
             param_grid={'C': array([1.00000000e-04, 2.04179592e-01, 4.08259184e-01, 6.12338776e-01,
       8.16418367e-01, 1.02049796e+00, 1.22457755e+00, 1.42865714e+00,
       1.63273673e+00, 1.83681633e+00, 2.04089592e+00, 2.24497551e+00,
       2.44905510e+00, 2.65313469e+00, 2.85721429e+...
       5.71432857e+00, 5.91840816e+00, 6.12248776e+00, 6.32656735e+00,
       6.53064694e+00, 6.73472653e+00, 6.93880612e+00, 7.14288571e+00,
       7.34696531e+00, 7.55104490e+00, 7.75512449e+00, 7.95920408e+00,
       8.16328367e+00, 8.36736327e+00, 8.57144286e+00, 8.77552245e+00,
       8.97960204e+00, 9.18368163e+00, 9.38776122e+00, 9.59184082e+00,
       9.79592041e+00, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [74]:
y_pred = grid_search.predict_proba(df_emb_test_)

In [ ]:
predictions0=[]
predictions1=[]


for res in y_pred:
    predictions0.append(res[0])
    predictions1.append(res[1])

    
test_data['predict_0']=predictions0
test_data['predict_1']=predictions1

test_data_predict=TestCutoff(test_data)

In [76]:
precision, recall, f1score = precision_recall_fscore_support(labels_test, test_data_predict['predict'])[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.85514834 0.81818182], recall: [0.92105263 0.69485294], f1score: [0.88687783 0.75149105]


In [77]:
precision, recall, f1score = precision_recall_fscore_support(labels_test, test_data_predict['predict'], average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score_macro: {f1score}')

precision: 0.8366650801205775, recall: 0.807952786377709, f1score_macro: 0.8191844408661155
